Unzip file from datasets.

In [10]:
from py7zr import SevenZipFile

with SevenZipFile("datasets/CHIvTOR.7z", "r") as zip:
    json_file = zip.getnames()[0]
    zip.extractall(path="data/")

Read json data

In [11]:
import json

f = open(f"data/{json_file}")
data = json.load(f)
f.close()

Prepare training and test data

In [13]:
from classes import Player

events = data["events"]

def getTeam():
    # extracting player ids of the home team
    team = {x["playerid"]: (f"{x['firstname']} {x['lastname']}", x["jersey"]) for x in events[0]["home"]["players"]}
    # only considering players that are starting the game
    starting_team = []
    for player in events[0]["moments"][0][5]:
        if player[1] in team.keys():
            starting_team.append(Player(player[1], team[player[1]][0], team[player[1]][1]))
    return starting_team

train_data, test_data = getTeam(), getTeam()

for event in events:
    # not considering time outs or quarter breaks
    if len(event["moments"]) == 0:
        continue
    # ignoring free throws
    if event["moments"][0][3] is None:
        continue
    quarter, ball_x, player_pos = event["moments"][0][0], event["moments"][0][5][0][2], event["moments"][0][5]
    # only considering defensive plays (using ball position)
    # first half
    if quarter < 3 and ball_x < 50:
        continue
    # second half
    if quarter > 2 and ball_x > 50:
        continue
    
    # player positions for the first 3 quarters will be used for training and last quarter for testing
    if quarter <= 3:
        for player in train_data:
            for tid, pid, px, py, pz in player_pos:
                if player.id == pid:
                    player.addPosition(px, py)
                    break
    else:
        for player in test_data:
            for tid, pid, px, py, pz in player_pos:
                if player.id == pid:
                    player.addPosition(px, py)
                    break